In [1]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable 
from tqdm.notebook import tqdm



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [4]:
data_path = '/content/drive/MyDrive/비트트레이더/데이터'
train_x_df = pd.read_csv(data_path  + "/train_x_df.csv")
train_y_df = pd.read_csv(data_path  + "/train_y_df.csv")
test_x_df = pd.read_csv(data_path  + "/test_x_df.csv")

In [5]:
train_y_df

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,9,0.999389,0.999630,0.999261,0.999385,0.001187,9.632785,0.012565,0.000605,4.904978
1,0,1,9,0.999593,1.000000,0.999265,0.999266,0.004197,34.052364,0.020572,0.003014,24.453999
2,0,2,9,0.999266,0.999618,0.999262,0.999322,0.003942,31.981472,0.016630,0.002967,24.069899
3,0,3,9,0.999322,0.999388,0.998917,0.999387,0.003663,29.709074,0.019463,0.002362,19.158766
4,0,4,9,0.999015,0.999015,0.997631,0.998399,0.003266,26.466251,0.030180,0.001533,12.421005
...,...,...,...,...,...,...,...,...,...,...,...,...
919315,7660,115,8,1.002662,1.003382,1.002606,1.002717,0.957148,173.131668,0.510148,0.476201,86.143784
919316,7660,116,8,1.002551,1.002606,1.001608,1.001830,1.537946,277.939087,0.465787,1.061331,191.801056
919317,7660,117,8,1.001830,1.002384,1.001497,1.001608,1.203197,217.364487,0.637684,0.971337,175.472198
919318,7660,118,8,1.001608,1.001941,1.001109,1.001386,1.252859,226.264069,1.003660,0.449899,81.251137


## Data preprocessing

input : (seq_len, batch, input_size)

In [6]:
X_train_tensors = torch.Tensor(np.array(train_x_df.iloc[:, [3, 6]]))

X_test_tensors = torch.Tensor(np.array(test_x_df.iloc[:, [3, 6]]))

y_train_tensors = torch.Tensor(np.array(train_y_df.iloc[:, 6]))

# X_train_tensors = Variable(torch.Tensor(np.array(train_x_df[train_x_df.loc[:, col_name[3:]]])))

# X_test_tensors = Variable(torch.Tensor(np.array(test_x_df[test_x_df.loc[:, col_name[3:]]])))

# y_train_tensors = Variable(torch.Tensor(np.array(train_y_df[train_y_df.iloc[:, 3:])))

In [7]:
#reshaping to rows, timestamps, features

X_train_tensors_final = torch.reshape(X_train_tensors,   (-1, 1380, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (-1 , 1380, X_test_tensors.shape[1])) 
y_train_tensors_final =torch.reshape(y_train_tensors,  (-1, 120)) 
# X_train_tensors_final = torch.reshape(train_x_df.iloc[:,3].values,   (-1, 1380, 1))


In [8]:
X_test_tensors_final.shape

torch.Size([535, 1380, 2])

In [9]:
X_train_tensors_final.shape

torch.Size([7661, 1380, 2])

In [10]:
y_train_tensors_final.shape

torch.Size([7661, 120])

In [11]:
len(X_train_tensors_final)

7661

In [12]:
n_iters = 1800
batch_size = 18
num_epochs = n_iters / (len(X_train_tensors_final) / batch_size)

train = torch.utils.data.TensorDataset(X_train_tensors_final,y_train_tensors_final)
test = torch.utils.data.TensorDataset(X_test_tensors_final)

train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

# batch_size_1 = 23

# test_loader = torch.utils.data.DataLoader(dataset=test, 
#                                           batch_size=batch_size_1, 
#                                           shuffle=False)

In [13]:
for i, (X_data, y_data) in enumerate(train_loader):
    print(X_data.shape, y_data.shape)

torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18, 120])
torch.Size([18, 1380, 2]) torch.Size([18

In [14]:
# X_train_data_loader = torch.utils.data.DataLoader(X_train_tensors_final, 32, shuffle=False)
# y_train_data_loader = torch.utils.data.DataLoader(y_train_tensors_final, 32, shuffle=False)
# X_test_data_loader = torch.utils.data.DataLoader(X_test_tensors_final, 32, shuffle=False)


In [15]:
#####################
input_dim = 2
hidden_dim = 32
num_layers = 1
output_dim = 120


class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[-1, 120, :]) 
        # out.size() --> 100, 10
        return out

In [16]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.1) 
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

LSTM(
  (lstm): LSTM(2, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=120, bias=True)
)
6
torch.Size([128, 2])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([120, 32])
torch.Size([120])


In [17]:
# Number of steps to unroll
seq_dim = 120 

for epoch in tqdm(range(int(num_epochs))):
    for i, (X_data, y_data) in enumerate(train_loader):
        # Load images as a torch tensor with gradient accumulation abilities
        #X_data = X_data.view(-1, seq_dim, input_dim).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(X_data)
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, y_data)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        if epoch % 100 == 0:
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([18, 120])) that is different to the input size (torch.Size([120])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 0.97869
Epoch: 0, loss: 0.54920
Epoch: 0, loss: 2.46681
Epoch: 0, loss: 0.77230
Epoch: 0, loss: 0.36390
Epoch: 0, loss: 0.21882
Epoch: 0, loss: 0.12833
Epoch: 0, loss: 0.06910
Epoch: 0, loss: 0.04854
Epoch: 0, loss: 0.03684
Epoch: 0, loss: 0.01805
Epoch: 0, loss: 0.00893
Epoch: 0, loss: 0.01063
Epoch: 0, loss: 0.01355
Epoch: 0, loss: 0.01340
Epoch: 0, loss: 0.01277
Epoch: 0, loss: 0.00894
Epoch: 0, loss: 0.00681
Epoch: 0, loss: 0.00569
Epoch: 0, loss: 0.00476
Epoch: 0, loss: 0.00418
Epoch: 0, loss: 0.00435
Epoch: 0, loss: 0.00340
Epoch: 0, loss: 0.00279
Epoch: 0, loss: 0.00242
Epoch: 0, loss: 0.00129
Epoch: 0, loss: 0.00094
Epoch: 0, loss: 0.00074
Epoch: 0, loss: 0.00075
Epoch: 0, loss: 0.00087
Epoch: 0, loss: 0.00121
Epoch: 0, loss: 0.00110
Epoch: 0, loss: 0.00118
Epoch: 0, loss: 0.00101
Epoch: 0, loss: 0.00086
Epoch: 0, loss: 0.00075
Epoch: 0, loss: 0.00073
Epoch: 0, loss: 0.00047
Epoch: 0, loss: 0.00038
Epoch: 0, loss: 0.00067
Epoch: 0, loss: 0.00031
Epoch: 0, loss: 

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([11, 120])) that is different to the input size (torch.Size([120])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 0.00006



In [18]:
y_train_pred = np.array([])

for _ , x_test_1 in enumerate(X_test_tensors_final):
    # print(x_test_1.size())
    x_test_2 = x_test_1.view(-1, 1380, 2)
    #print(x_test_2.size())
    y_train_pred=np.append(y_train_pred, model(x_test_2).data.numpy())

In [19]:
y_train_pred=pd.DataFrame(y_train_pred.reshape(-1, 120))

In [20]:
y_train_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119
0,1.003279,1.001761,1.007023,1.001409,1.001314,1.004388,1.004161,1.002859,1.009860,1.007068,1.002910,1.003690,1.004326,1.001595,1.007733,1.004176,1.008490,1.001032,1.003060,1.007409,1.011620,1.002883,1.005451,0.999616,1.002252,1.002649,1.002307,0.999027,1.001209,1.001178,0.999104,1.001645,1.001697,1.001841,1.001022,1.002744,0.998908,1.002399,0.999990,1.005194,...,1.001764,1.004372,1.006371,1.001625,1.000149,1.001818,1.001121,0.998878,0.999540,1.001458,1.001132,1.002967,1.004293,1.001403,1.005062,1.000873,1.005071,1.001723,1.003353,1.006135,1.002492,1.001989,1.006524,1.001498,0.999609,1.000891,1.001317,1.000169,0.999244,1.000871,1.004018,1.005140,1.005740,1.000271,1.001921,1.003548,1.002847,1.003528,1.002863,1.002832
1,1.003771,1.003095,1.003670,1.000403,1.001036,1.005098,1.003655,1.003256,1.006289,1.004676,1.005311,1.003663,1.003010,1.000701,1.006078,1.003687,1.005669,1.003210,1.002075,1.005744,1.009782,1.004278,1.003378,1.000897,1.002543,1.001579,1.001670,0.999387,1.000455,1.000923,1.000045,1.001942,1.003249,1.002028,1.001966,1.002399,0.999603,1.000496,1.000559,1.004055,...,1.001405,1.004018,1.005917,1.002290,1.001471,1.000014,1.002639,1.000444,1.000735,1.002491,0.999976,1.001012,1.001660,1.000260,1.000743,1.002821,1.003843,0.999721,1.001673,1.004378,1.002236,1.004179,1.004356,1.000078,1.000391,0.999495,0.998133,1.000632,0.999134,1.001613,1.000738,1.004110,1.003122,1.000628,1.003711,1.000539,1.005794,1.001819,1.001638,1.000378
2,1.003535,1.002324,1.005805,1.001064,1.001262,1.004771,1.004060,1.003083,1.008608,1.006248,1.003913,1.003779,1.003890,1.001304,1.007247,1.004048,1.007509,1.001962,1.002741,1.006929,1.011079,1.003523,1.004766,1.000170,1.002480,1.002331,1.002145,0.999217,1.000989,1.001130,0.999506,1.001852,1.002407,1.002030,1.001465,1.002717,0.999234,1.001734,1.000281,1.004852,...,1.001688,1.004362,1.006363,1.001978,1.000723,1.001205,1.001820,0.999540,1.000060,1.001953,1.000757,1.002310,1.003388,1.001029,1.003478,1.001696,1.004741,1.001036,1.002805,1.005616,1.002498,1.002907,1.005841,1.001029,0.999965,1.000416,1.000154,1.000417,0.999253,1.001233,1.002858,1.004878,1.004854,1.000486,1.002709,1.002446,1.004071,1.002955,1.002468,1.001953
3,1.003193,1.001603,1.007311,1.001482,1.001310,1.004258,1.004157,1.002780,1.010139,1.007247,1.002630,1.003635,1.004414,1.001652,1.007809,1.004189,1.008709,1.000767,1.003123,1.007482,1.011708,1.002687,1.005588,0.999455,1.002158,1.002698,1.002324,0.998967,1.001245,1.001175,0.998991,1.001561,1.001483,1.001759,1.000887,1.002720,0.998809,1.002547,0.999895,1.005250,...,1.001766,1.004334,1.006322,1.001505,0.999984,1.001947,1.000908,0.998694,0.999390,1.001300,1.001204,1.003105,1.004489,1.001477,1.005440,1.000642,1.005110,1.001873,1.003462,1.006220,1.002457,1.001733,1.006649,1.001594,0.999504,1.000991,1.001594,1.000086,0.999225,1.000756,1.004284,1.005168,1.005927,1.000192,1.001690,1.003809,1.002508,1.003651,1.002940,1.003037
4,1.002962,1.001232,1.007871,1.001603,1.001266,1.003911,1.004084,1.002560,1.010640,1.007557,1.001981,1.003446,1.004554,1.001744,1.007860,1.004183,1.009103,1.000142,1.003217,1.007514,1.011775,1.002201,1.005794,0.999075,1.001878,1.002729,1.002311,0.998808,1.001271,1.001136,0.998728,1.001325,1.000954,1.001506,1.000545,1.002598,0.998571,1.002820,0.999647,1.005304,...,1.001739,1.004161,1.006097,1.001187,0.999587,1.002165,1.000377,0.998258,0.999036,1.000894,1.001309,1.003326,1.004822,1.001589,1.006182,1.000101,1.005089,1.002143,1.003619,1.006292,1.002308,1.001136,1.006803,1.001750,0.999241,1.001165,1.002138,0.999864,0.999149,1.000464,1.004774,1.005132,1.006242,0.999972,1.001132,1.004326,1.001724,1.003853,1.003057,1.003435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [21]:
submission = pd.DataFrame(np.zeros([535, 2], np.int64),
            columns = ['buy_quantity', 'sell_time'])
submission = submission.reset_index()
submission.loc[:, 'buy_quantity'] = 0.1
selltime=[]
sell_price=[]
for idx, i in enumerate(range(y_train_pred.shape[0])):
    selltime.append(np.argmax(y_train_pred.iloc[i]))
    if np.argmax(y_train_pred.iloc[i])==2:
        #sell_price.append(y_train_pred.iloc[i][1])
        sell_price.append(1)
    else:
        sell_price.append(y_train_pred.iloc[i].min())

In [ ]:
submission.loc[:, 'buy_quantity']=sell_price
submission.loc[:, 'sell_time']=selltime

submission.to_csv(data_path+"/submission.csv", index = False)

submission


- 참조 : https://cnvrg.io/pytorch-lstm/?gclid=CjwKCAjwgOGCBhAlEiwA7FUXknvaSTYr_3NHdJqAuctM0aDmets4r7K9YyLoHXhe41Bh6RRooaYxYRoCuxAQAvD_BwE 
https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_lstm_neuralnetwork/
